# LoL-Predicting-Duration

**Name(s)**: Nick Liu & Tanisha Dighe

**Website Link**: https://tdighe2001.github.io/LoL-Predicting-Duration/

## Code

In [318]:
import pandas as pd
import numpy as np
import os

import plotly.express as px
pd.options.plotting.backend = 'plotly'

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import QuantileTransformer, label_binarize, Binarizer

### Framing the Problem

In [31]:
df = pd.read_csv(os.path.join('data', '2022_matches.csv'))
df.head()

/Users/nick/opt/anaconda3/envs/dsc80/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3460: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
0,ESPORTSTMNT01_2690210,complete,NaN,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,121.0,391.0,345.0,14.0,0.0,1.0,0.0,0.0,1.0,0.0
1,ESPORTSTMNT01_2690210,complete,NaN,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,100.0,541.0,-275.0,-11.0,2.0,3.0,2.0,0.0,5.0,1.0
2,ESPORTSTMNT01_2690210,complete,NaN,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,119.0,-475.0,153.0,1.0,0.0,3.0,0.0,3.0,3.0,2.0
3,ESPORTSTMNT01_2690210,complete,NaN,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,149.0,-793.0,-1343.0,-34.0,2.0,1.0,2.0,3.0,3.0,0.0
4,ESPORTSTMNT01_2690210,complete,NaN,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,21.0,443.0,-497.0,7.0,1.0,2.0,2.0,0.0,6.0,2.0


In [335]:
print(cleaned.head().to_markdown(index=False))

| gameid                | league   |   year |   playoffs |   game |   patch |   gamelength |   result |   kills |   deaths |   assists |   teamkills |   teamdeaths |   doublekills |   triplekills |   quadrakills |   pentakills |   firstblood |   team kpm |   ckpm |   firstdragon |   dragons |   opp_dragons |   infernals |   mountains |   clouds |   oceans |   elders |   opp_elders |   firstherald |   heralds |   opp_heralds |   firstbaron |   barons |   opp_barons |   firsttower |   towers |   opp_towers |   firsttothreetowers |   inhibitors |   opp_inhibitors |   damagetochampions |     dpm |   damagetakenperminute |   damagemitigatedperminute |   wardsplaced |     wpm |   wardskilled |    wcpm |   controlwardsbought |   visionscore |    vspm |   totalgold |   earnedgold |   earned gpm |   goldspent |   gspd |   minionkills |   monsterkills |    cspm |   goldat10 |   xpat10 |   csat10 |   opp_goldat10 |   opp_xpat10 |   opp_csat10 |   golddiffat10 |   xpdiffat10 |   csdiffat10 |   kil

In [310]:
#remove all columns and rows that have nans and select team columns
newdf = df[df["datacompleteness"] == "complete"]
newdf = newdf[newdf["position"] == "team"]
newdf = newdf.dropna(axis='columns')
newdf = newdf.drop(['participantid','position', 'side', 'datacompleteness'], axis = 1)
cleaned = newdf.groupby(['gameid', 'league']).mean().reset_index()
cleaned.head()

,gameid,league,year,playoffs,game,patch,gamelength,result,kills,deaths,...,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
0,ESPORTSTMNT01_2690210,LCK CL,2022.0,0.0,1.0,12.01,1713.0,0.5,14.0,14.0,...,498.5,0.0,0.0,0.0,5.5,14.0,5.5,5.5,14.0,5.5
1,ESPORTSTMNT01_2690219,LCK CL,2022.0,0.0,1.0,12.01,2114.0,0.5,9.5,9.5,...,544.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0
2,ESPORTSTMNT01_2690227,LCK CL,2022.0,0.0,1.0,12.01,1972.0,0.5,9.5,9.5,...,552.5,0.0,0.0,0.0,2.0,4.5,2.0,2.0,4.5,2.0
3,ESPORTSTMNT01_2690255,LCK CL,2022.0,0.0,1.0,12.01,2488.0,0.5,14.5,14.5,...,537.0,0.0,0.0,0.0,3.0,8.0,3.0,3.0,8.0,3.0
4,ESPORTSTMNT01_2690264,LCK CL,2022.0,0.0,1.0,12.01,2020.0,0.5,10.5,10.5,...,538.5,0.0,0.0,0.0,3.0,5.0,3.0,3.0,5.0,3.0


In [259]:
#TRAIN-TEST SPLIT
X = cleaned.drop('gamelength', axis = 1)
y = cleaned['gamelength']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

### Baseline Model

In [314]:
def baseline_pipeline(X_train, y_train):

    categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder())
    ])

    preprocessor = ColumnTransformer(
        transformers=[
        ('cat', categorical_transformer,['league'])
    ], remainder='passthrough')

    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', LinearRegression())
    ])


    X = X_train[['league', 'goldspent']]
    y = y_train



    return pipeline.fit(X, y)
basepipeline = baseline_pipeline(X_train, y_train)
basepreds = basepipeline.predict(X_test)
basermse = mean_squared_error(y_test, basepreds, squared=False)
baser2 = basepipeline.score(X_test, y_test)
baser2, basermse

(0.9354008998515442, 84.83291987751124)

### Final Model

In [265]:
class StdScalerByGroup(BaseEstimator, TransformerMixin):
    # performs group-wise standardization.

    def __init__(self):
        pass

    def fit(self, X, y=None):
        # X might not be a pandas DataFrame (e.g. a np.array)
        df = pd.DataFrame(X)

        # Compute and store the means/standard-deviations for each column (e.g. 'c1' and 'c2'), 
        # for each group (e.g. 'A', 'B', 'C').  
        # (Our solution uses a dictionary)
        stds = X.groupby(X[X.columns[0]]).std()
        means = X.groupby(X[X.columns[0]]).mean()
        self.grps_ = (means.to_dict(), stds.to_dict())


        return self

    def transform(self, X, y=None):

        try:
            getattr(self, "grps_")
        except AttributeError:
            raise RuntimeError("You must fit the transformer before tranforming the data!")
        
        lol = pd.DataFrame()
        df = pd.DataFrame(X)
        data = df.copy()
        dic = self.grps_
        for g in data.iloc[:, 0].unique():
            new = data[data.iloc[:, 0] == g].iloc[:, 1:].apply(lambda x: (x-dic[0][x.name][g])/dic[1][x.name][g])
            old_ind = new.index
            new = new.set_index([old_ind, [g]*len(new)])
            lol = pd.concat([lol, new], axis = 0)
        lol = lol.sort_index()
        return lol

In [266]:
def final_pipeline(X_train, y_train, model):

    categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(sparse = False))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
        ('cat', categorical_transformer, ['league']),
        ('std-group', StdScalerByGroup(), ['monsterkills']),
        ('quant', QuantileTransformer(), ['damagetochampions'])
    ], remainder='passthrough')

    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])


    X = X_train[['patch','league', 'goldspent', 'monsterkills', 'damagetochampions']]
    y = y_train



    return pipeline.fit(X, y)

In [313]:
finalpipeline = final_pipeline(X_train, y_train, LinearRegression())
finpreds = finalpipeline.predict(X_test)
finrmse = mean_squared_error(y_test, finpreds, squared=False)
finr2 = finalpipeline.score(X_test, y_test)
finrmse = mean_squared_error(finalpipeline.predict(X_test), y_test, squared=False)
finr2, finrmse

(0.9364469625245184, 84.14326060715081)

In [279]:
# Choosing the best model
regressors = [
    GradientBoostingRegressor(),
    KNeighborsRegressor(),
    ExtraTreesRegressor(),
    RandomForestRegressor(),
    DecisionTreeRegressor(),
    LinearRegression(),
    Lasso(),
    Ridge(),
    MLPRegressor()
]

In [336]:
for model in regressors:
    mdl = final_pipeline(X_train, y_train, model)
    y_pred = mdl.predict(X_test) 
    print(model)
    print("R2 score:", r2_score(y_test, y_pred))
    print("RMSE score:", mean_squared_error(y_test, y_pred, squared=False))
    print()

GradientBoostingRegressor()
R2 score: 0.9361511035346678
RMSE score: 84.33888960756258

KNeighborsRegressor()
R2 score: 0.9195096912368506
RMSE score: 94.69411243746968

ExtraTreesRegressor()
R2 score: 0.9247372252574966
RMSE score: 91.5674866685892

RandomForestRegressor()
R2 score: 0.9293604036052424
RMSE score: 88.71055282906973

DecisionTreeRegressor()
R2 score: 0.873848300425188
RMSE score: 118.54895609234104

LinearRegression()
R2 score: 0.9364469625245184
RMSE score: 84.14326060715081

Lasso()
R2 score: 0.9343451657121927
RMSE score: 85.52331686879154

Ridge()
R2 score: 0.9364678880044545
RMSE score: 84.12940695718684

MLPRegressor()
R2 score: 0.9219613989159879
RMSE score: 93.24078450911317



In [253]:
#Finding best hyperparameters/model

In [250]:
parameters = {'regressor__learning_rate': [0.01,0.02,0.03,0.04],
              'regressor__subsample'    : [0.9, 0.5, 0.2, 0.1],
              'regressor__n_estimators' : [100,500,1000, 1500],
              'regressor__max_depth'    : [4,6,8,10]
             }

In [251]:
grid_GBR = GridSearchCV(estimator=final_pipeline(X_train, y_train, GradientBoostingRegressor()), param_grid = parameters, cv = 2, n_jobs=-1)
grid_GBR.fit(X_train, y_train)


GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('cat',
                                                                         Pipeline(steps=[('onehot',
                                                                                          OneHotEncoder(sparse=False))]),
                                                                         ['league']),
                                                                        ('std-group',
                                                                         StdScalerByGroup(),
                                                                         ['monsterkills']),
                                                                        ('quant',
                                                                         QuantileTransformer(),
     

In [274]:
parameters = {'regressor__fit_intercept':[True,False],
              'regressor__normalize':[True,False],
              'regressor__copy_X':[True, False]}

In [275]:
grid_lin = GridSearchCV(estimator=final_pipeline(X_train, y_train, LinearRegression()), param_grid=parameters, cv = 2, n_jobs=-1)
grid_lin.fit(X_train, y_train)

/Users/nick/opt/anaconda3/envs/dsc80/lib/python3.8/site-packages/sklearn/linear_model/_base.py:141: FutureWarning:

'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)





GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('cat',
                                                                         Pipeline(steps=[('onehot',
                                                                                          OneHotEncoder(sparse=False))]),
                                                                         ['league']),
                                                                        ('std-group',
                                                                         StdScalerByGroup(),
                                                                         ['monsterkills']),
                                                                        ('quant',
                                                                         QuantileTransformer(),
     

In [292]:
parameters = {'regressor__solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga', 'lbfgs'],
              'regressor__alpha': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}

In [293]:
grid_ridge = GridSearchCV(estimator=final_pipeline(X_train, y_train, Ridge()), param_grid=parameters, cv = 2, n_jobs=-1)
grid_ridge.fit(X_train, y_train)

/Users/nick/opt/anaconda3/envs/dsc80/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning:


20 fits failed out of a total of 160.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/nick/opt/anaconda3/envs/dsc80/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/nick/opt/anaconda3/envs/dsc80/lib/python3.8/site-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/nick/opt/anaconda3/envs/dsc80/lib/python3.8/site-packages/sklearn/linear_mod

GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('cat',
                                                                         Pipeline(steps=[('onehot',
                                                                                          OneHotEncoder(sparse=False))]),
                                                                         ['league']),
                                                                        ('std-group',
                                                                         StdScalerByGroup(),
                                                                         ['monsterkills']),
                                                                        ('quant',
                                                                         QuantileTransformer(),
     

In [324]:
gbrr2 = grid_GBR.score(X_test, y_test)
gbrrmse = mean_squared_error(grid_GBR.predict(X_test), y_test, squared=False)
gbrscore = (gbrr2, gbrrmse)

linr2 = grid_lin.score(X_test, y_test)
linrmse = mean_squared_error(grid_lin.predict(X_test), y_test, squared=False)
linscore = (linr2, linrmse)

ridger2 = grid_ridge.score(X_test, y_test)
ridgermse = mean_squared_error(grid_ridge.predict(X_test), y_test, squared=False)
ridgescore = (ridger2, ridgermse)

In [325]:
gbrscore

(0.9479765316465001, 76.12912400221188)

In [326]:
linscore

(0.9372775172564793, 83.59163145993799)

In [327]:
ridgescore

(0.9364678880044545, 84.12940695718684)

In [332]:
finr2, finrmse

(0.9364469625245184, 84.14326060715081)

In [333]:
baser2, basermse

(0.9354008998515442, 84.83291987751124)

In [328]:
#best params

In [329]:
grid_GBR.best_params_

{'regressor__learning_rate': 0.01,
 'regressor__max_depth': 4,
 'regressor__n_estimators': 1500,
 'regressor__subsample': 0.5}

### Fairness Analysis

In [ ]:
#Null Hypothesis: Our model is fair. Its RMSE for LCK CL and other leagues are roughly the same, 
#and any differences are due to random chance.
#Alternative Hypothesis: Our model is unfair. Its RMSE for LCK CL league is lower than 
#its precision for other leagues

In [337]:
#OBSERVED STAT
finpreds = finalpipeline.predict(X_test)
finr2 = finalpipeline.score(X_test, y_test)

obsdf = pd.DataFrame()

binarizer = Binarizer()
X_test_league = binarizer.fit_transform(X_test['league'].array.reshape(-1,1) == 'VCS').flatten()

obsdf['X_test_league'] = pd.Series(X_test_league)
obsdf['y_test'] = y_test.array
obsdf['preds'] = finpreds
obsdf

,X_test_league,y_test,preds
0,False,1832.0,1799.771978
1,False,1760.0,1980.479900
2,False,2399.0,2303.956355
3,False,2285.0,2260.856406
4,False,2072.0,2077.784654
...,...,...,...
2650,False,1775.0,1742.406219
2651,False,2130.0,2057.594095
2652,False,1826.0,1837.713113
2653,False,1567.0,1674.732688


In [338]:
(
    obsdf
    .groupby('X_test_league')
    .apply(lambda x: mean_squared_error(obsdf['y_test'], obsdf['preds'], squared=False))
    .rename('RMSE')
    .to_frame()
)
obs = obsdf.groupby('X_test_league').apply(lambda x: mean_squared_error(obsdf['y_test'], obsdf['preds'], squared=False)).diff().iloc[-1]
obs

0.0

In [339]:
diff_in_rmse = []
for _ in range(1000):
    s = (
        obsdf[['X_test_league', 'preds', 'y_test']]
        .assign(X_test_league=obsdf.X_test_league.sample(frac=1.0, replace=False).reset_index(drop=True))
        .groupby('X_test_league')
        .apply(lambda x: mean_squared_error(x['y_test'], x['preds'], squared=False))
        .diff()
        .iloc[-1]
    )
    
    diff_in_rmse.append(s)

In [340]:
p_value = (diff_in_rmse <= obs).mean()
p_value
#pval is greater than 0.05 so fair to reject null huypothesis. our model is fair!

0.535